In [169]:
import pandas as pd
import os
from task1 import create_csv1

create columns of DataFrame

In [170]:
def create_columns(name_class_good: str, name_class_bad: str, df: pd.DataFrame):
    df['review'] = ""
    df['number_words'] = ""
    i = 0
    for file_dataset in os.listdir("dataset"):
        path_p = os.path.join("dataset", file_dataset)
        class_name = os.listdir(path_p)
        for elem in class_name:
            name = os.path.join(path_p, elem)
            f = open(name, mode = 'rt', encoding = 'utf-8')
            data = f.read()
            df.loc[[i], 'review'] = data 
            words = data.split()
            df.loc[[i], 'number_words'] = len(words)
            f.close()
            i += 1

create new DataFrame with limiting value

In [204]:
def create_new_df(df: pd.DataFrame, limit: int):
    return df[df['number_words'] <= limit][['class_name', 'review', 'number_words']]

create new DataFrame by label

In [172]:
def create_df_metka(df: pd.DataFrame, metka: str):
    return df[df['class_name'] == metka][['class_name', 'review', 'number_words']]

In [208]:
def group_df(df: pd.DataFrame):
    print("average: ", df.number_words.mean(axis=0))
    print("maximum: ", df.number_words.max())
    print("minimum", df.number_words.min())
    return df.groupby('class_name').count() 

create DataFrame

In [ ]:
name_class_good = "good"
name_class_bad = "bad"
create_csv1(name_class_bad, name_class_good, "annotation")
df = pd.read_csv("annotation", encoding = 'utf-16', delimiter=';',names=['absolut_path', 'otnos_path', 'class_name'])
df = df.drop(['absolut_path', 'otnos_path'], axis=1)
df

checking for emptiness

In [ ]:
pd.isnull(df)

counting number of empty elements

In [ ]:
df.isnull().sum()

In [ ]:
create_columns(name_class_good, name_class_bad, df)
df

In [ ]:
df_lim = create_new_df(df, 100)
df_lim

In [ ]:
df_met = create_df_metka(df, name_class_good)
df_met

In [209]:
df_group = group_df(df)
df_group

average:  369.6215
maximum:  1791
minimum 35


,review,number_words
class_name,,
bad,1000,1000
good,1000,1000
